## Loading Packages & Initialization

In [1]:
rm(list=ls())

library(data.table)
library(tidyverse)
library(rJava)
library(RNetLogo)
library(lhs)

options(warn = 0)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
Warning message:
“package ‘tibble’ was built under R version 3.6.3”Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Warning message:
“package ‘dplyr’ was built under R version 3.6.3”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.tab

In [2]:
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"
source(paste0(folder.path,"ThesisSetupCode.r"))

Is_Headless <- 1
nl.model <- "info_cascade_update_TDP_JPF_2020" #"Segregation_Dummy"

nl.path <- "/Users/ecemnaz.yildiz/Documents/NetLogo 6.0.4/Java"
folder.path = "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/"

model.path <- paste0(folder.path, nl.model, ".nlogo")

if (Is_Headless == 0) {
    NLStart(nl.path, gui = TRUE, nl.jarname = "netlogo-6.0.4.jar")
    NLLoadModel(model.path)
} else {
    NLStart(nl.path, gui = FALSE, nl.jarname = "netlogo-6.0.4.jar", nl.obj = nl.model)
    NLLoadModel(model.path, nl.obj = nl.model)
}


Attaching package: ‘plotly’

The following object is masked from ‘package:igraph’:

    groups

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

The following object is masked from ‘package:ggplot2’:

    margin

Warning message:
“package ‘factoextra’ was built under R version 3.6.3”Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa
Loading required package: gridExtra
Warning message:
“package ‘gridExtra’ was built under R version 3.6.3”
Attaching package: ‘gridExtra’

The following object is maske

In [3]:
model.type = "info_cascade_update" ##ifelse(nl.model == "Segregation", "basic", "dummy")
# the path of data folder
data.path = paste0(folder.path,"data/")
# the path for outputs to be record
output.folder = paste0("outputs_V3_RFE_mtrymultip2_",model.type,"_",Sys.Date())
dir.create(file.path(folder.path, output.folder), showWarnings = FALSE)

outputs.path = paste0(folder.path,output.folder,"/")

# Read Me File to keep info about the output folder
ReadMe = paste0(outputs.path,"ReadMe_",model.type,".txt")

## Model Parameters & Functions

### Set model parameters

In [4]:
#### Model Parameters #### Set model parameters Number of replications for each
#### instance
nofrep = 30 #############################

# order feature names according to their definition order in run_model

    feature_names = c("ground-truth","max_links"
                  ,"evidence","sc-bel-prop"
                  ,"prop-likelihood","p-h-given-c"
                  ,"n_init_believers","learning_rate"
                  ,"con-threshold","expertise_influence") 
    feature_ranges = data.table(  feature   = feature_names
                                , min_range = c(0, 2,   0,   0, 0, 0, 0,   0, 0, 0)
                                , max_range = c(1, 500, 100, 5, 1, 1, 100, 1, 1, 1))

# 
output_name = c("cl-prop-same")

# Number of input parameters of the agent-based model
nofparams = length(feature_names)

# set RF parameters
ntree = 300
#mtry = 2
mtry.multiplier = 2 # when 1, it is default, when 2, it is at most twice of defaults 
nperm = 5

### Set user parameters

In [5]:
#### User parameters ####
error_type = "RMSE"  # MAPE, BIAS

# choose the uncertainty measure
selection_metric <- "coefvar"  #, 'range' 
sample.type = paste0("AdFe_",selection_metric)

elimination.type = "NRFE" # or "RFE"

# Number of iterations
iteration_budget = 9
metarep = c(1:3)#c(1:10)

# Number of instances
unlabeled_ins = 30
test_ins = 30 ##c(100,400)
train_ins_oneshot = 30
train_ins_Ad = 30 ##50

# Set selection parameters
selected_ins = 5  #nofinstancesWillbeSelected in each step

# Set elimination parameter
p = 0.8 # elimination proportion
# h = 1
oob_allowance = 0.1

seed.focus = 0 ##c(1,2,3,4,5,6,7,8,9,20)

## !!!
unlabeled.type = "refresh and ElimInducedSampling"

# Decide on strategy:
elimination_start_iter = 5

log_entry()

## Test Set

In [6]:
#### Test Sets ####
test_set = data.table()
for( t in test_ins){
    test_set.name= paste0(data.path,"test_set","_",model.type,"_",t,".csv")
    test_set_Sub <- fread(test_set.name)  
    
    test_set = rbind(test_set, data.table(size = t, test_set_Sub))
    
    #assign(paste0("test_set_",t),test_set)
}

In [7]:
paste0(data.path,"test_set","_",model.type,"_",test_ins,".csv")

[1] "/Users/ecemnaz.yildiz/Documents/Personal/Thesis/data/test_set_info_cascade_update_30.csv"

In [ ]:
test_set

## Adaptive Training Set

In [7]:
adaptive_initial_data = upload_training_set(model.type,seed.focus,train_ins_Ad)

In [ ]:
adaptive_initial_data

### Adaptive & Feature Elimination Train & Test Metamodel

In [ ]:
sample.type = paste0("AdFe_",selection_metric)
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

for(i in seed.focus){ print(paste0("seed : ",i,"  Adaptive Sampling with Feature Selection section start time : ",Sys.time()))    
    for (r in metarep){ print(paste0("seed : ", i,"   rep : ", r, "  Adaptive Sampling with Feature Selection section start time : ", Sys.time()))
        set.seed(i + r)
            
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])
        train_candidates_table = data.table()
        
        columns_left = feature_names # reset at the beginning of each iteration
        total_numof_eliminated_vars <- 0 # reset at the beginning of each iteration
        eliminated_columns = c()
    
        iteration_history = data.table("seed" = integer(),"rep" = integer(),"iter_no" = integer()
                              ,"IsFeatureEliminated" = logical(), "IsDataSelected" = logical()
                              ,"NumOfEliminated" = integer(), "RankedUpd" = logical())
        iter = 1
        while(iter <= iteration_budget){   
            print(iter)
            run_log_entry()
    
            trainx = training_set_Ad[,.SD, .SDcols = columns_left]
            trainy = training_set_Ad$output
        
            run_step_log_entry("Model Training Start.")
            
            # Train the model
            model_Sub <- randomForest( x = trainx, y =  trainy,importance = TRUE
                                      ,ntree = ntree, nperm = nperm
                                      ,mtry = mtry_default(columns_left) * mtry.multiplier)
                model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
                model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                saveRDS(model_Sub, model_Sub.path)
        
            iteration_history= rbind(iteration_history,data.table(i,r,iter,0,0,0,0), use.names = FALSE)
            # update VIM or not
            if (elimination.type == "RFE" | (elimination.type == "NRFE" & (length(columns_left) == length(feature_names)))){
                ranked_features = get_variable_importance(model_Sub)
                iteration_history[iter]$RankedUpd= 1 
                
            }     
       
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type
        
            if(iter != iteration_budget){ # below efforts are unnecessary when the budget is reached. 
                
                run_step_log_entry("Sample Selection Start.")
         
                ### SAMPLE SELECTION ###    
                #select samples first but not to add to the training set until eliminated_features are specified.
                # select new data candidates before elimination
                ## sample selection from unlabeled data select candidates
                unlabeled_set <- refresh_sample_pool(i + r + iter, columns_left)
                train_candidates = sample_selection(selected_ins, unlabeled_set, model_Sub,selection_metric)
                
                run_step_log_entry("ABM Run Start.")
                
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ",Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                
                run_step_log_entry("ABM Run End.")
                
                print(paste0("ABM train_candidate run end time : ",Sys.time()))
                
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )      

                ### SAMPLE SELECTION ENDS ###
                
                ### FEATURE ELIMINATION ###
                if(elimination_start_iter <= iter & length(columns_left) >= 2){ #######ilk deneylerde eşitlik yoktu.
                    check_elim = TRUE 
                    apply_elim = FALSE
                    # 
                ### FEATURE ELIMINATION PART I ###
                #decide how many features will be eliminated
                    elim_check_iter = 1
                    h = floor(length(columns_left) * (p^elim_check_iter))
                    while(check_elim){
                    
                        run_step_log_entry("Feature Selection Start.")
    
                        # Assume as if feature(s) will be eliminated
                        feature_elimination_result = feature_elimination(h, columns_left, ranked_features)
                        planned_columns_left = feature_elimination_result[[1]]
                        
                        run_step_log_entry("New Random Forest Model Generation.")
                        
                        model_Sub_afterElim <- randomForest(  x = training_set_Ad[,.SD, .SDcols = planned_columns_left]
                                                             ,y =  training_set_Ad$output
                                                             ,importance = TRUE, nperm = nperm
                                                             ,ntree = ntree
                                                            , mtry = mtry_default(planned_columns_left) * mtry.multiplier)        
                            model_Sub_afterElim.name = paste0("model_afterElim_",sample.type,"_", iter, "_seed_", i, "_rep_",r,"_h_",h)
                            model_Sub_afterElim.path = paste0(outputs.path,models.folder, paste0(model_Sub_afterElim.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                            saveRDS(model_Sub_afterElim, model_Sub_afterElim.path)

                        run_step_log_entry("New Random Forest Model OOB Calculation.")

                        new_oob = obb_error_func(model_Sub_afterElim)
                    
                        if(new_oob < (obb_err + obb_err * oob_allowance)){ 

                            run_step_log_entry("New Random Forest Model Selected.")

                            check_elim = FALSE 
                            apply_elim = TRUE                            

                        } else {
                            
                            run_step_log_entry("New Random Forest Model is not Selected.")

                            elim_check_iter = elim_check_iter + 1
                            h_upd = floor(length(columns_left) * (p^elim_check_iter)) 
                            
                            if(h_upd == h){ # if h does not change
                                check_elim = FALSE    
                            }
                            
                            h = copy(h_upd)
                        }
                     }             
               ### FEATURE SELECTION PART II ###
               # really eliminate 
                    if(apply_elim){
                        
                        run_step_log_entry("Feature Elimination Applied.")
                        
                        # update iteration_history
                        iteration_history[iter]$IsFeatureEliminated= 1
                        iteration_history[iter]$NumOfEliminated= length(columns_left) - length(planned_columns_left)
                
                        columns_left = planned_columns_left
                        eliminated_columns =  feature_elimination_result[[4]]

                        run_step_log_entry("Eliminated Columns Recorded.")
                    
                    }         
               }
              ### FEATURE SELECTION ENDS ###
            
              # add labeled candidates to the train data
              training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
              # update iteration_history
              iteration_history[iter]$IsDataSelected= 1

                run_step_log_entry("Labeled Data Added to Training Set.")
  
            }
            fwrite(iteration_history[iter],paste0(outputs.path,sample.folder,model.type,"_iteration_history_",sample.type,".csv"),append = TRUE )       

            run_step_log_entry(paste0("Iteration history Updated. Iteration ", iter, " Ends."))

            iter = iter + 1

        }
        
        run_step_log_entry("Final Train Data File Recorded.")
    
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry("Eliminated Columns File Recorded.")

        fwrite(data.table("seed" = i,"rep" = r, "elim_cols" =  eliminated_columns),paste0(outputs.path,sample.folder,model.type,"_EliminatedColumns_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))       
                       
    ##    print(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    }
    
    run_step_log_entry(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))                 
                     
##    print(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    #rm(training_set_Ad,predictedLabels_table,train_candidates_table)      
}

In [8]:
sample.type = paste0("AdFe_",selection_metric)
sample.folder = paste0(sample.type,"/")
dir.create(file.path(folder.path, output.folder,sample.folder), showWarnings = FALSE)

models.folder = paste0("models_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,models.folder), showWarnings = FALSE)

PL.folder = paste0("PL_",sample.type,"/")
dir.create(file.path(folder.path, output.folder,PL.folder), showWarnings = FALSE)

i = 0
    
    print(paste0("seed : ",i,"  Adaptive Sampling with Feature Selection section start time : ",Sys.time()))    
r = 2 
        print(paste0("seed : ", i,"   rep : ", r, "  Adaptive Sampling with Feature Selection section start time : ", Sys.time()))
        set.seed(i + r)
            
        training_set_Ad = copy(adaptive_initial_data[seed == i, .SD, .SDcols = -c("seed")])
        train_candidates_table = data.table()
        
        columns_left = feature_names # reset at the beginning of each iteration
        total_numof_eliminated_vars <- 0 # reset at the beginning of each iteration
        eliminated_columns = c()
    
        iteration_history = data.table("seed" = integer(),"rep" = integer(),"iter_no" = integer()
                              ,"IsFeatureEliminated" = logical(), "IsDataSelected" = logical()
                              ,"NumOfEliminated" = integer(), "RankedUpd" = logical())
        iter = 1
        while(iter <= iteration_budget){   
            print(iter)
            run_log_entry()
    
            trainx = training_set_Ad[,.SD, .SDcols = columns_left]
            trainy = training_set_Ad$output
        
            run_step_log_entry("Model Training Start.")
            
            # Train the model
            model_Sub <- randomForest( x = trainx, y =  trainy,importance = TRUE
                                      ,ntree = ntree, nperm = nperm
                                      ,mtry = mtry_default(columns_left) * mtry.multiplier)
                model_Sub.name = paste0("model_",sample.type,"_", iter, "_seed_", i, "_rep_",r)
                model_Sub.path = paste0(outputs.path,models.folder, paste0(model_Sub.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                saveRDS(model_Sub, model_Sub.path)
        
            iteration_history= rbind(iteration_history,data.table(i,r,iter,0,0,0,0), use.names = FALSE)
            # update VIM or not
            if (elimination.type == "RFE" | (elimination.type == "NRFE" & (length(columns_left) == length(feature_names)))){
                ranked_features = get_variable_importance(model_Sub)
                iteration_history[iter]$RankedUpd= 1 
                
            }     
       
            # write errors 
            obb_err = obb_error_func(model_Sub)     
            fwrite(data.table(iter,obb_error = obb_err,seed = i,rep = r)
                   ,paste0(outputs.path,sample.folder,model.type,"_","obb_error_",sample.type,".csv") ,append = TRUE)
        
            write_test_accuracy(i,r,iter,model_Sub,test_set, error_type)
            write_importance.rf(i,r,iter,model_Sub,sample.type)#last one=sample_type
        
            if(iter != iteration_budget){ # below efforts are unnecessary when the budget is reached. 
                
                run_step_log_entry("Sample Selection Start.")
         
                ### SAMPLE SELECTION ###    
                #select samples first but not to add to the training set until eliminated_features are specified.
                # select new data candidates before elimination
                ## sample selection from unlabeled data select candidates
                unlabeled_set <- refresh_sample_pool(i + r + iter, columns_left)
                train_candidates = sample_selection(selected_ins, unlabeled_set, model_Sub,selection_metric)
                
                fwrite(data.table(setcolorder(train_candidates, feature_names), "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_ABM_train_candidates_table_"
                       ,sample.type,".csv"),append = TRUE )      


                run_step_log_entry("ABM Run Start.")
                
                # run ABM to find outputs of train candidates
                print(paste0("ABM train_candidate run start time : ",Sys.time()))
                train_candidates = run_ABM(nofrep, selected_ins, train_candidates)
                
                run_step_log_entry("ABM Run End.")
                
                print(paste0("ABM train_candidate run end time : ",Sys.time()))
                
                fwrite(data.table(train_candidates, "iter" = iter, "seed" = i, "rep" = r)
                       ,paste0(outputs.path,sample.folder,model.type,"_train_candidates_table_",sample.type,".csv"),append = TRUE )      

                ### SAMPLE SELECTION ENDS ###
                
                ### FEATURE ELIMINATION ###
                if(elimination_start_iter <= iter & length(columns_left) >= 2){ #######ilk deneylerde eşitlik yoktu.
                    check_elim = TRUE 
                    apply_elim = FALSE
                    # 
                ### FEATURE ELIMINATION PART I ###
                #decide how many features will be eliminated
                    elim_check_iter = 1
                    h = floor(length(columns_left) * (p^elim_check_iter))
                    while(check_elim){
                    
                        run_step_log_entry("Feature Selection Start.")
    
                        # Assume as if feature(s) will be eliminated
                        feature_elimination_result = feature_elimination(h, columns_left, ranked_features)
                        planned_columns_left = feature_elimination_result[[1]]
                        
                        run_step_log_entry("New Random Forest Model Generation.")
                        
                        model_Sub_afterElim <- randomForest(  x = training_set_Ad[,.SD, .SDcols = planned_columns_left]
                                                             ,y =  training_set_Ad$output
                                                             ,importance = TRUE, nperm = nperm
                                                             ,ntree = ntree
                                                            , mtry = mtry_default(planned_columns_left) * mtry.multiplier)        
                            model_Sub_afterElim.name = paste0("model_afterElim_",sample.type,"_", iter, "_seed_", i, "_rep_",r,"_h_",h)
                            model_Sub_afterElim.path = paste0(outputs.path,models.folder, paste0(model_Sub_afterElim.name,"_size_",train_ins_Ad, ".rds"))  # to save the model
                            saveRDS(model_Sub_afterElim, model_Sub_afterElim.path)

                        run_step_log_entry("New Random Forest Model OOB Calculation.")

                        new_oob = obb_error_func(model_Sub_afterElim)
                    
                        if(new_oob < (obb_err + obb_err * oob_allowance)){ 

                            run_step_log_entry("New Random Forest Model Selected.")

                            check_elim = FALSE 
                            apply_elim = TRUE                            

                        } else {
                            
                            run_step_log_entry("New Random Forest Model is not Selected.")

                            elim_check_iter = elim_check_iter + 1
                            h_upd = floor(length(columns_left) * (p^elim_check_iter)) 
                            
                            if(h_upd == h){ # if h does not change
                                check_elim = FALSE    
                            }
                            
                            h = copy(h_upd)
                        }
                     }             
               ### FEATURE SELECTION PART II ###
               # really eliminate 
                    if(apply_elim){
                        
                        run_step_log_entry("Feature Elimination Applied.")
                        
                        # update iteration_history
                        iteration_history[iter]$IsFeatureEliminated= 1
                        iteration_history[iter]$NumOfEliminated= length(columns_left) - length(planned_columns_left)
                
                        columns_left = planned_columns_left
                        eliminated_columns =  feature_elimination_result[[4]]

                        run_step_log_entry("Eliminated Columns Recorded.")
                    
                    }         
               }
              ### FEATURE SELECTION ENDS ###
            
              # add labeled candidates to the train data
              training_set_Ad = rbind(training_set_Ad, train_candidates[, -c("idx")],use.names = TRUE)
              # update iteration_history
              iteration_history[iter]$IsDataSelected= 1

                run_step_log_entry("Labeled Data Added to Training Set.")
  
            }
            fwrite(iteration_history[iter],paste0(outputs.path,sample.folder,model.type,"_iteration_history_",sample.type,".csv"),append = TRUE )       

            run_step_log_entry(paste0("Iteration history Updated. Iteration ", iter, " Ends."))

            iter = iter + 1

        }
        
        run_step_log_entry("Final Train Data File Recorded.")
    
        fwrite(data.table(training_set_Ad, "seed" = i,"rep" = r),paste0(outputs.path,sample.folder,model.type,"_FinalTrainData_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry("Eliminated Columns File Recorded.")

        fwrite(data.table("seed" = i,"rep" = r, "elim_cols" =  eliminated_columns),paste0(outputs.path,sample.folder,model.type,"_EliminatedColumns_",sample.type,".csv") ,append = TRUE)

        run_step_log_entry(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))       
                       
    ##    print(paste0("seed : ",i,"   rep : ", r,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    
    
    run_step_log_entry(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))                 
                     
##    print(paste0("seed : ",i,"  Adaptive Sampling with Feature Elimination section end time : ",Sys.time()))
    #rm(training_set_Ad,predictedLabels_table,train_candidates_table)      


[1] "seed : 0  Adaptive Sampling with Feature Selection section start time : 2021-12-18 14:50:32"
[1] "seed : 0   rep : 2  Adaptive Sampling with Feature Selection section start time : 2021-12-18 14:50:32"
[1] 1
[1] "ABM train_candidate run start time : 2021-12-18 14:50:33"
[1] "ABM train_candidate run end time : 2021-12-18 14:52:41"
[1] 2
[1] "ABM train_candidate run start time : 2021-12-18 14:52:41"
[1] "ABM train_candidate run end time : 2021-12-18 14:54:54"
[1] 3
[1] "ABM train_candidate run start time : 2021-12-18 14:54:54"
[1] "ABM train_candidate run end time : 2021-12-18 15:00:25"
[1] 4
[1] "ABM train_candidate run start time : 2021-12-18 15:00:25"
[1] "ABM train_candidate run end time : 2021-12-18 15:04:30"
[1] 5
[1] "ABM train_candidate run start time : 2021-12-18 15:04:30"
[1] "ABM train_candidate run end time : 2021-12-18 15:08:51"
[1] 6
[1] "ABM train_candidate run start time : 2021-12-18 15:08:52"
[1] "ABM train_candidate run end time : 2021-12-18 15:11:04"
[1] 7
[1] "ABM

## Quit NL

In [ ]:
NLQuit(nl.obj = nl.model)